In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [22]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [23]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [24]:
train_df=train_df.drop('Cabin',axis=1)
train_df=train_df.dropna()

In [25]:
num=['Age','Fare']
num_feature=Pipeline(steps=[
    ("scaler",StandardScaler())
])


In [26]:
test_df['Embarked']=test_df['Embarked'].replace(test_df['Embarked'].unique(),[0,1,2])
train_df['Embarked']=train_df['Embarked'].replace(train_df['Embarked'].unique(),[0,1,2])


In [27]:
test_df['Sex']=test_df['Sex'].replace(test_df['Sex'].unique(),[0,1])
train_df['Sex']=train_df['Sex'].replace(train_df['Sex'].unique(),[0,1])

In [28]:
test_df=test_df.drop("Cabin",axis=1)

In [29]:
test_df['Age']=test_df['Age'].fillna(test_df['Age'].median())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].median())

In [30]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
train_df['IsAlone'] = (train_df['FamilySize'] == 1)

In [31]:
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1
test_df['IsAlone'] = (test_df['FamilySize'] == 1)

In [32]:
# nor=['Pclass','Sex','Embarked']
# nor_feature=Pipeline(steps=[
#     ("encoder",OneHotEncoder(handle_unknown="ignore"))
# ])

In [33]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    int64  
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    int64  
 11  FamilySize   712 non-null    int64  
 12  IsAlone      712 non-null    bool   
dtypes: bool(1), float64(2), int64(8), object(2)
memory usage: 73.0+ KB
None


In [34]:
processor=ColumnTransformer(transformers=[
    ("num_transform",num_feature,num),
    ],
    remainder='passthrough'
)

In [35]:
features=['Age','Fare','Pclass','Sex','Embarked','FamilySize',"IsAlone"]

In [41]:
train_df[features]

,Age,Fare,Pclass,Sex,Embarked,FamilySize,IsAlone
0,22.0,7.2500,3,0,0,2,False
1,38.0,71.2833,1,1,1,2,False
2,26.0,7.9250,3,1,0,1,True
3,35.0,53.1000,1,1,0,2,False
4,35.0,8.0500,3,0,0,1,True
...,...,...,...,...,...,...,...
885,39.0,29.1250,3,1,2,6,False
886,27.0,13.0000,2,0,0,1,True
887,19.0,30.0000,1,1,0,1,True
889,26.0,30.0000,1,0,1,1,True


In [36]:
x_train=processor.fit_transform(train_df[features])
x_test=processor.transform(test_df[features])
y_train=train_df['Survived']

In [37]:
x_train


array([[-0.527668557291984, -0.5163799235734484, 3, ..., 0, 2, False],
       [0.5770938751757179, 0.6940460475663703, 1, ..., 1, 2, False],
       [-0.25147794917505856, -0.5036203511241232, 3, ..., 0, 1, True],
       ...,
       [-0.7348115133796782, -0.08633507435544971, 1, ..., 0, 1, True],
       [-0.25147794917505856, -0.08633507435544971, 1, ..., 1, 1, True],
       [0.16280796300032965, -0.5069283884258001, 3, ..., 2, 1, True]],
      dtype=object)

In [38]:
model = SVC(kernel='rbf', C=1, gamma='scale')
model.fit(x_train,y_train)

SVC(C=1)

In [39]:
y_predict=model.predict(x_test)

In [40]:
submission = pd.DataFrame({"PassengerId": test_df["PassengerId"], "Survived": y_predict})

submission.to_csv("submission.csv", index=False)